In [28]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import tqdm

import matplotlib.pyplot as plt


import numpy as np
%matplotlib qt

In [29]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [30]:
class ChorusNeuralNetworkDataset(torch.utils.data.Dataset):
    
    def __init__(self, dataset_version, is_validation = False):
        
        self.dataset_version = dataset_version

        CONJUNCTIONS_REFS = np.load(f"./../processed_data_chorus_neural_network/STAGE_4/{dataset_version}/MODEL_READY_DATA_{dataset_version}.npz")

        if is_validation:
            
            self.features = CONJUNCTIONS_REFS["VALIDATION_FEATURES"].astype(np.float32)
            self.labels = CONJUNCTIONS_REFS["VALIDATION_LABELS"].astype(np.float32)
        else:
            
            self.features = CONJUNCTIONS_REFS["FEATURES"].astype(np.float32)
            self.labels = CONJUNCTIONS_REFS["LABELS"].astype(np.float32)
                    
        CONJUNCTIONS_REFS.close()
        
    def get_bins(self):
        
        return self.BINS
        
    # get sample
    def __getitem__(self, idx):
        
        return torch.from_numpy(self.features[idx, :]), torch.from_numpy(self.labels[idx, :])

    def __len__(self):
        return int(self.labels.shape[0])

In [31]:
# Define model
class ChorusNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.RELU_STACK_THEN_LINEAR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(15, 1024),
            nn.PReLU(),
            nn.Linear(1024, 256),
            nn.PReLU(),
            nn.Linear(256, 1),
        )

    def forward(self, x):
        logits = self.RELU_STACK_THEN_LINEAR(x)
        return logits

model = ChorusNeuralNetwork().to(device)
print(model)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in model: {total_params}")

ChorusNeuralNetwork(
  (RELU_STACK_THEN_LINEAR): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=15, out_features=1024, bias=True)
    (2): PReLU(num_parameters=1)
    (3): Linear(in_features=1024, out_features=256, bias=True)
    (4): PReLU(num_parameters=1)
    (5): Linear(in_features=256, out_features=1, bias=True)
  )
)
Total number of parameters in model: 279043


In [32]:
dataset = ChorusNeuralNetworkDataset("v4a")
validation_set = ChorusNeuralNetworkDataset("v4a", is_validation=True)

BATCH_SIZE = 64

print(f"Training set size: {len(dataset)}")
print(f"Validation set size: {len(validation_set)}")


train_dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
)

validation_dataloader = torch.utils.data.DataLoader(
            validation_set,
            batch_size=BATCH_SIZE,
            shuffle=True,
)

Training set size: 345313
Validation set size: 9138


In [33]:
loss_fn = nn.SmoothL1Loss("mean")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.75)


In [34]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        
        optimizer.zero_grad()
        
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"training loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [35]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0.0
        
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
    test_loss /= num_batches
    return test_loss

In [36]:
num_epochs = 100
training_losses = []
validation_losses = []
for epoch in range(num_epochs):
    
    print(f"Maximum learning rate: {scheduler.get_last_lr()[0]}")
    
    print(f"Beginning training for epoch: {epoch}!")
    training_loss = train(train_dataloader, model, loss_fn, optimizer)
    training_losses.append(training_loss)
    
    print(f"Beginning validation testing for epoch: {epoch}!")
    validation_loss = test(validation_dataloader, model, loss_fn)
    print(f"Validation Error: \n Avg loss: {validation_loss:>8f} \n")
    validation_losses.append(validation_loss)
    
    torch.save(model.state_dict(), os.path.abspath(f"./../processed_data_chorus_neural_network/TRAINED_MODELS/Regression_with_smaller_dataset/Regression_with_smaller_dataset_{epoch}.pth"))
    print(f"Saved PyTorch Model State to {os.path.abspath(f"./../processed_data_chorus_neural_network/TRAINED_MODELS/Regression_with_smaller_dataset/Regression_with_smaller_dataset_{epoch}.pth")}")
    
    scheduler.step()

Maximum learning rate: 0.001
Beginning training for epoch: 0!
training loss: 0.606920  [   64/345313]
training loss: 0.136775  [64064/345313]
training loss: 0.198092  [128064/345313]
training loss: 0.267768  [192064/345313]
training loss: 0.158098  [256064/345313]
training loss: 0.149851  [320064/345313]
Beginning validation testing for epoch: 0!
Validation Error: 
 Avg loss: 0.210179 

Saved PyTorch Model State to c:\Dev\Research\REPT_Enhancements_Tool\processed_data_chorus_neural_network\TRAINED_MODELS\Regression_with_smaller_dataset\Regression_with_smaller_dataset_0.pth
Maximum learning rate: 0.00075
Beginning training for epoch: 1!
training loss: 0.218964  [   64/345313]
training loss: 0.250755  [64064/345313]
training loss: 0.215892  [128064/345313]
training loss: 0.216524  [192064/345313]
training loss: 0.155179  [256064/345313]
training loss: 0.173043  [320064/345313]
Beginning validation testing for epoch: 1!
Validation Error: 
 Avg loss: 0.210551 

Saved PyTorch Model State to

KeyboardInterrupt: 

In [43]:
plt.plot(validation_losses)
plt.plot(training_losses)

In [10]:
recent_model = os.path.abspath("./../processed_data_chorus_neural_network/TRAINED_MODELS/Regression_with_smaller_dataset/Regression_with_smaller_dataset_12.pth")

#torch.save(model.state_dict(), recent_model)
print(f"Saved PyTorch Model State to {recent_model}")    

Saved PyTorch Model State to c:\Dev\Research\REPT_Enhancements_Tool\processed_data_chorus_neural_network\TRAINED_MODELS\Regression_with_smaller_dataset\Regression_with_smaller_dataset_12.pth


In [11]:
model = ChorusNeuralNetwork().to(device)
model.load_state_dict(torch.load(recent_model, weights_only=True))

model.eval()

with torch.no_grad():
    x, y = torch.tensor(validation_set.features), validation_set.labels
    x = x.to(device)
    pred = np.array(model(x).cpu())


In [12]:
plt.scatter(np.exp(validation_set.labels), np.exp(pred))
plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-0, 1e2)
plt.ylim(1e-0, 1e2)

(1.0, 100.0)

In [14]:
model.eval()

with torch.no_grad():
    x, y = torch.tensor(dataset.features), dataset.labels
    x = x.to(device)
    pred = np.array(model(x).cpu())


plt.scatter(np.exp(dataset.labels), np.exp(pred))
plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-1, 1e2)
plt.ylim(1e-1, 1e2)

(0.1, 100.0)